<a href="https://colab.research.google.com/github/nikitakandpal/ML-projects/blob/main/TopicModellingLDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
import praw
import pandas as pd
from datetime import datetime, timedelta

# Initialize Reddit API client using PRAW
reddit = praw.Reddit(client_id='iHJMBdnO5iGHhBkXxixt0Q',
                     client_secret='-0K1jhtSUfGrEECt0p7YcANMGM6rwQ',
                     user_agent='test',
                     username='smmtest',
                     password='SMMclass22!!')

In [ ]:
post_id = '1cbqdgp'
num_comments = 120

post = reddit.submission(id=post_id)

# Ensure all comments are loaded, not just the top-level ones
post.comments.replace_more(limit=None)
comments = post.comments.list()[:num_comments]

data = []

for comment in comments:
    subreddit_name = post.subreddit.display_name
    author = comment.author.name if comment.author else '[deleted]'
    content = comment.body
    # Convert UTC timestamp to MM/DD/YYYY format
    date = datetime.utcfromtimestamp(comment.created_utc).strftime('%m/%d/%Y')  # Changed to MM/DD/YYYY
    score = comment.score
    post_title = post.title  # Get the title of the post
    data.append([author, content, post_title, subreddit_name, date, score])

# Create a pandas DataFrame from the data
df = pd.DataFrame(data, columns=['Author', 'Content', 'Post Title', 'Subreddit', 'Date', 'Score'])

# Write the DataFrame to a CSV file
df.to_csv('reddit_data.csv', index=False)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from collections import Counter
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.models import CoherenceModel
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
import pandas as pd

# List of potential encodings to try
encodings = ['latin1', 'ISO-8859-1', 'cp1252', 'utf-16']

for enc in encodings:
    try:
        data = pd.read_csv("politics22analysis.csv", encoding=enc, quotechar='"', escapechar="\\", on_bad_lines='skip')
        print(f"Data loaded successfully with encoding {enc}.")
        break
    except UnicodeDecodeError as e:
        print(f"Encoding {enc} did not work: {e}")
    except Exception as e:
        print(f"An unexpected error occurred with encoding {enc}: {e}")


Data loaded successfully with encoding latin1.


<ipython-input-2-f8472decc903>:8: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("politics22analysis.csv", encoding=enc, quotechar='"', escapechar="\\", on_bad_lines='skip')


In [3]:
df = pd.DataFrame(data)

# Remove all columns between column index 1 to 3
df.drop(df.iloc[:, 7:], inplace=True, axis=1)

In [4]:
df

,author,content,date,tag,title,url,website
0,The Quint,"Tulsi Gabbard, the first Hindu lawmaker from H...",13/11/22,Tulsi Gabbard,Tulsi Gabbard: Hindu War Veteran US Lawmaker M...,https://www.thequint.com/news/world/tulsi-gabb...,quint
1,Pushpesh Pant,Swami Vivekananda ranks among the highest in t...,03/07/22,"Swami Vivekananda,Hinduism,Tolerance,Politics ...",Swami VivekanandaÃ¢ÂÂs 156th Birth Anniversa...,https://www.thequint.com/voices/opinion/swami-...,quint
2,Meeran Chadha Borwankar,"In the news for the wrong reasons, the Central...",18/07/22,"Modi Government,Central Bureau of Investigatio...",Rift Between CBI Chief & Special Director: Thi...,https://www.thequint.com/voices/opinion/cbi-co...,quint
3,Mayukh Ghosh,Reminiscing about the past is a dreadful tool ...,14/05/22,"Satyajit Ray,Shyam Benegal,Smita Patil,Politic...",The Late Filmmaker Mrinal SenÃ¢ÂÂs Ã¢ÂÂCin...,https://www.thequint.com/entertainment/indian-...,quint
4,Akanksha Kumar,: Akanksha Kumar : Sandeep Suman Twenty-three ...,30/04/22,"Tihar Jail,Supreme Court of India,Life Impriso...",Tandoor Murder: Former Delhi Police Joint Comm...,https://www.thequint.com/videos/news-videos/ho...,quint
...,...,...,...,...,...,...,...
10644,Aditi,Review petitions have been filed before the Su...,17/11/22,"Kerala,kerala government,Sabarimala,Sabarimala...",Sabarimala: Dawn to dusk bandh in Kerala as Co...,https://www.opindia.com/2018/11/sabarimala-daw...,opindia
10645,OpIndia Staff,"In a new statement, Congress Minister from Pun...",17/11/22,"Congress,congress gaffes,Narendra Modi,navjot ...",PM Modi is jealous of him getting invited to P...,https://www.opindia.com/2018/11/pm-modi-is-jea...,opindia
10646,OpIndia Staff,The demand comes a day after the backward clas...,17/11/22,"devendra fadnavis,Maharashtra,Maratha Reservat...",Muslim leaders write to Maharashtra CM Devendr...,https://www.opindia.com/2018/11/muslim-leaders...,opindia
10647,OpIndia Staff,Surjewala states through the Punjab state gove...,17/11/22,"Amit Shah,BJP,Congress,navjot singh sidhu,Rand...",CongressÃ¢ÂÂ Randeep Surjewala writes to HM ...,https://www.opindia.com/2018/11/congress-rande...,opindia


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to preprocess the text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

# Apply the preprocessing to the content column
df['processed_content'] = df['content'].dropna().apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
#df.to_csv('processed_data.csv', index=False)

In [7]:
from gensim import corpora, models

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(df['processed_content'].dropna())

# Filter out words that occur less than 20 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=20, no_above=0.5)

# Convert dictionary to a Bag of Words (BoW) corpus
corpus = [dictionary.doc2bow(text) for text in df['processed_content'].dropna()]

In [8]:
# Set the number of topics
num_topics = 5

# Create the LDA model
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Print the topics found by the LDA model
topics = lda_model.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.014*"assembly" + 0.014*"seats" + 0.013*"elections" + 0.013*"election"')
(1, '0.009*"political" + 0.007*"one" + 0.006*"people" + 0.006*"would"')
(2, '0.012*"rs" + 0.011*"india" + 0.010*"modi" + 0.008*"crore"')
(3, '0.013*"court" + 0.011*"police" + 0.007*"case" + 0.006*":"')
(4, '0.009*"leader" + 0.008*"modi" + 0.008*"people" + 0.008*"sabha"')


In [9]:
pip install pyldavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.8 MB/s eta 0:00:00


In [10]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
pyLDAvis.enable_notebook()
lda_display = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

# Display the visualization
pyLDAvis.display(lda_display)